In [30]:
import os
import requests
import pandas as pd
import pdfplumber
from io import BytesIO
import logging
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sqlalchemy import create_engine
 
# Load environment variables
load_dotenv()

# Retrieve credentials and paths from .env file
DB_USERNAME = os.getenv('DB_USERNAME')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_DSN = os.getenv('DB_DSN')
CHROME_DRIVER_PATH = os.getenv('CHROME_DRIVER_PATH')
 
connection = cx_Oracle.connect(username, password, dsn, mode=cx_Oracle.SYSDBA)
 
table_name = 'abl_data'
 
# Assuming df_except_jurisdiction is your DataFrame
if not final.empty:
    engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{dsn}?mode=SYSDBA', max_identifier_length=128)
 
    # Use the engine directly as the 'con' parameter for to_sql
    final.to_sql(table_name, engine, index=False)
 

COUNTRIES DATAFRAME

In [5]:
# Your list of countries
countries = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas", "Balkans", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Burma",
    "Benin", "Bhutan", "Bolivia", "Bosnia & Herzegovina", "Botswana","Brazil", "Brunei", "Bulgaria", "Burkina Faso", "Burundi", "Cabo Verde","Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad""Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Cayman Island",  # Corrected spelling
    "Croatia", "Cuba", "Cyprus", "Czech Republic", "Denmark", "Djibouti", "Hong Kong",
    "Dominica", "Dominican Republic", "Ecuador", "Egypt", "El Salvador", 
    "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini", "Ethiopia", 
    "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", 
    "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Gibraltar",
    "Guyana", "Haiti", "Honduras", "Hungary", "Iceland", "India", 
    "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Jamaica", 
    "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "Korea, North", 
    "Korea, South", "Kosovo", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", 
    "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", 
    "Luxembourg", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", 
    "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", 
    "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro", 
    "Morocco", "Mozambique", "Myanmar", "Namibia", "Nauru", "Nepal", 
    "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Korea",
    "North Macedonia", "Norway", "Oman", "Pakistan", "Palau", 
    "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", 
    "Poland", "Portugal", "Qatar", "Romania", "Russia", "Rwanda", 
    "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", 
    "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", 
    "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Sudan",  # Corrected spelling
    "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", 
    "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Sweden", 
    "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania","The United Arab Emirates",  # Added comma
    "Thailand", "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago", 
    "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", 
    "United Arab Emirates", "United Kingdom", "United States", 
    "Uruguay", "Uzbekistan", "Vanuatu", "Vatican City", 
    "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe", "Democratic People's Republic of Korea (DPRK – North Korea)", "Democratic Republic of the Congo", "Myanmar (Burma)"
]

# Create the DataFrame
df_countries = pd.DataFrame(countries, columns=['Country'])

# Remove whitespaces
df_countries['Country'] = df_countries['Country'].str.strip()

# Print the cleaned DataFrame
print(df_countries)


                                               Country
0                                          Afghanistan
1                                              Albania
2                                              Algeria
3                                              Andorra
4                                               Angola
..                                                 ...
199                                             Zambia
200                                           Zimbabwe
201  Democratic People's Republic of Korea (DPRK – ...
202                   Democratic Republic of the Congo
203                                    Myanmar (Burma)

[204 rows x 1 columns]


In [6]:
display(df_countries)

,Country
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
199,Zambia
200,Zimbabwe
201,Democratic People's Republic of Korea (DPRK – ...
202,Democratic Republic of the Congo


COLOUMN 3 

In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import os
import urllib.request
import urllib.parse

# Function to scrape active sanction programs
def scrape_active_sanction_programs(base_url):
    driver = webdriver.Chrome()

    try:
        driver.get(base_url)

        program_links = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.XPATH, '//table[@class="table cols-2"]//tbody//tr//td[1]/a'))
        )

        data = [] # List to store data for DataFrame

        for link in program_links:
            program_name = link.text
            program_url = link.get_attribute("href")

            # Call the modified scrape_executive_orders function
            orders_data = scrape_executive_orders(program_url, program_name)
            data.extend(orders_data)

        # Create a DataFrame
        columns = ["Country", "Restrictive Measures", "Executive Orders", "Type of Sanctions"]
        df = pd.DataFrame(data, columns=columns)
        return df
        

    finally:
        # Close the webdriver
        driver.quit()

# Function to scrape executive orders
def scrape_executive_orders(url, program_name, heading_name='Sanction Brochures'):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        heading = soup.find('h3', string=lambda text: text and 'Sanctions Brochures' in text)

        orders_data = []  # List to store data for DataFrame

        if heading:
            ul_tag = heading.find_next('ul')

            if ul_tag:
                a_tag = ul_tag.find('a')

                if a_tag:
                    pdf_link = a_tag.get('href')
                    #first_url = 'https://ofac.treasury.gov/'
                    #full_url = first_url + pdf_link
                    full_url = urllib.parse.urljoin('https://ofac.treasury.gov/', pdf_link)
                    #download_pdf(full_url, program_name)
                    orders_data.append([program_name, "", f"\n•{full_url}", ""])

        
            heading1 = soup.find('h4', string=lambda text: text and 'Executive Orders' in text or 'h4')
            if heading1:
                ul_tag = heading1.find_next('ul')

                if ul_tag:
                    for li_tag in ul_tag.find_all('li'):
                        a_tag = li_tag.find('a')
                        if a_tag:
                            pdf_link = a_tag.get('href')
                            #full_url = 'https://ofac.treasury.gov' + pdf_link
                            full_url = urllib.parse.urljoin('https://ofac.treasury.gov/', pdf_link)
                            order_title = li_tag.text
                            #order_title = order_title + f"\n ● {order_title}"
                            #download_pdf(full_url, program_name)
                            orders_data.append([program_name, "", f"\n•{order_title} {full_url}", ""])
        else:
            heading1 = soup.find('h4', string=lambda text: text and 'Executive Orders' in text)
            if heading1:
                ul_tag = heading1.find_next('ul')

                if ul_tag:
                    for li_tag in ul_tag.find_all('li'):
                        a_tag = li_tag.find('a')
                        if a_tag:
                            pdf_link = a_tag.get('href')
                            #full_url = 'https://ofac.treasury.gov' + pdf_link
                            full_url = urllib.parse.urljoin('https://ofac.treasury.gov/', pdf_link)
                            order_title = li_tag.text
                            #order_title = order_title + f"\n ● {order_title}"
                            #download_pdf(full_url, program_name)
                            orders_data.append([program_name, "", f"\n•{order_title} {full_url}", ""])


    return orders_data

 
def download_pdf(url, filename):
    try:
        # Create the directory if it doesn't exist
        os.makedirs("pdfs", exist_ok=True)
        
        pdf_data = urllib.request.urlopen(url).read()
        with open(os.path.join("pdfs", f"{filename}.pdf"), 'wb') as pdf_file:
            pdf_file.write(pdf_data)
            print(f"PDF downloaded: {filename}.pdf")
 
    except Exception as e:
        print(f"Failed to download PDF: {e}")
 

# Base URL of the website
base_url = 'https://ofac.treasury.gov/sanctions-programs-and-country-information'

# Scrape data from active sanction programs and store it in df
df = scrape_active_sanction_programs(base_url)


In [8]:
df.style.format({'Executive Orders': '=HYPERLINK("{}")'})
display(df)

,Country,Restrictive Measures,Executive Orders,Type of Sanctions
0,Afghanistan-Related Sanctions,,\n•14064 Protecting Certain Property of Da Afg...,
1,Afghanistan-Related Sanctions,,\n•13268 Termination of Emergency With Respect...,
2,Afghanistan-Related Sanctions,,\n•13224 Blocking Property and Prohibiting Tra...,
3,Balkans-Related Sanctions,,\n•https://ofac.treasury.gov/media/6181/downlo...,
4,Balkans-Related Sanctions,,\n•14033 Blocking Property and Suspending Entr...,
...,...,...,...,...
146,Yemen-related Sanctions,,\n•13611 - Blocking Property of Persons Threat...,
147,Zimbabwe Sanctions,,\n•https://ofac.treasury.gov/media/7441/downlo...,
148,Zimbabwe Sanctions,,\n•13469 Blocking Property of Additional Perso...,
149,Zimbabwe Sanctions,,\n•13391 Blocking Property of Additional Perso...,


In [57]:
df.groupby('Country')['Executive Orders'].agg(lambda x: '\n'.join(x)).reset_index()
df['Restrictive Measures'] = None
df['Type of Sanctions'] = None

In [58]:
display(df)

,Country,Restrictive Measures,Executive Orders,Type of Sanctions
0,Afghanistan-Related Sanctions,None,\n•14064 Protecting Certain Property of Da Afg...,None
1,Afghanistan-Related Sanctions,None,\n•13268 Termination of Emergency With Respect...,None
2,Afghanistan-Related Sanctions,None,\n•13224 Blocking Property and Prohibiting Tra...,None
3,Balkans-Related Sanctions,None,\n•https://ofac.treasury.gov/media/6181/downlo...,None
4,Balkans-Related Sanctions,None,\n•14033 Blocking Property and Suspending Entr...,None
...,...,...,...,...
146,Yemen-related Sanctions,None,\n•13611 - Blocking Property of Persons Threat...,None
147,Zimbabwe Sanctions,None,\n•https://ofac.treasury.gov/media/7441/downlo...,None
148,Zimbabwe Sanctions,None,\n•13469 Blocking Property of Additional Perso...,None
149,Zimbabwe Sanctions,None,\n•13391 Blocking Property of Additional Perso...,None


In [65]:
df['Country'] = df['Country'].str.replace(r'-Related Sanctions| Sanctions|-|related|and Darfur|(Executive Order 13894 of 2019)|-related Sanctions|\(|\)', '', regex=True).str.rstrip()

In [66]:
df.assign(Country=df['Country'].str.split('/')).explode('Country')

,Country,Restrictive Measures,Executive Orders,Type of Sanctions
0,Afghanistan,None,\n•14064 Protecting Certain Property of Da Afg...,None
1,Balkans,None,\n•13268 Termination of Emergency With Respect...,None
2,Belarus,None,\n•13224 Blocking Property and Prohibiting Tra...,None
3,Burma,None,\n•https://ofac.treasury.gov/media/6181/downlo...,None
4,Central African Republic,None,\n•14033 Blocking Property and Suspending Entr...,None
...,...,...,...,...
146,NaN,None,\n•13611 - Blocking Property of Persons Threat...,None
147,NaN,None,\n•https://ofac.treasury.gov/media/7441/downlo...,None
148,NaN,None,\n•13469 Blocking Property of Additional Perso...,None
149,NaN,None,\n•13391 Blocking Property of Additional Perso...,None


In [67]:
df.groupby('Country')['Executive Orders'].agg(lambda x: '\n'.join(x)).reset_index()
df['Restrictive Measures'] = None
df['Type of Sanctions'] = None

In [68]:
display(df)

,Country,Restrictive Measures,Executive Orders,Type of Sanctions
0,Afghanistan,None,\n•14064 Protecting Certain Property of Da Afg...,None
1,Balkans,None,\n•13268 Termination of Emergency With Respect...,None
2,Belarus,None,\n•13224 Blocking Property and Prohibiting Tra...,None
3,Burma,None,\n•https://ofac.treasury.gov/media/6181/downlo...,None
4,Central African Republic,None,\n•14033 Blocking Property and Suspending Entr...,None
...,...,...,...,...
146,NaN,None,\n•13611 - Blocking Property of Persons Threat...,None
147,NaN,None,\n•https://ofac.treasury.gov/media/7441/downlo...,None
148,NaN,None,\n•13469 Blocking Property of Additional Perso...,None
149,NaN,None,\n•13391 Blocking Property of Additional Perso...,None


In [76]:
from sqlalchemy import create_engine
import cx_Oracle
import pandas as pd
 
username = 'sys'
password = 'sys123'
dsn = 'orcl'
 
connection = cx_Oracle.connect(username, password, dsn, mode=cx_Oracle.SYSDBA)
 
table_name = 'executive_order'
 
# Assuming df_except_jurisdiction is your DataFrame
if not result_df.empty:
    engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{dsn}?mode=SYSDBA', max_identifier_length=128)
 
    # Use the engine directly as the 'con' parameter for to_sql
    result_df.to_sql(table_name, engine, index=False)

COLOUMN 4

In [14]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pdfplumber
from io import BytesIO
import urllib.request
 
# Correct column names
columns_list = ["Country", "Restrictive Measures", "Executive Orders", "Type of Sanctions"]
 
# Initialize the DataFrame with the correct columns
security_council_report_df = pd.DataFrame(columns=columns_list)
 
start_keyword = 'Types of Sanctions'
end_keywords = ['Number of Listed Individuals', 'Exemptions']
 
# Initialize an empty list to store data
data_list = []
 
def extract_specific_information(text, start_keyword, end_keywords):
    start_index = text.find(start_keyword)
 
    if start_index != -1:
        # Find the end index for each keyword
        end_indices = [text.find(keyword, start_index) for keyword in end_keywords if keyword != -1]
 
        # Extract the text between start and the minimum end index
        min_end_index = min(end_index for end_index in end_indices if end_index != -1)
        if min_end_index != -1:
            return text[start_index + len(start_keyword):min_end_index]
        else:
            return text[start_index:]
 
    return None
 
def scrape_content_from_url(fact_sheet_href, start_keyword, end_keywords):
    try:
        # Set a user-agent in the headers to simulate a browser request
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
 
        # Download content from URL with headers
        response = requests.get(fact_sheet_href, headers=headers)
        response.raise_for_status()
 
        # Check the content type
        content_type = response.headers.get('content-type', '').lower()
 
        if 'application/pdf' in content_type:
            # Open the PDF from the downloaded content
            with pdfplumber.open(BytesIO(response.content)) as pdf:
                # Loop through all pages in the PDF
                for page_number in range(len(pdf.pages)):
                    page = pdf.pages[page_number]
 
                    # Extract text from the page
                    text = page.extract_text()
 
                    # Extract specific information based on criteria
                    specific_info = extract_specific_information(text, start_keyword, end_keywords)
 
                    # If information is found, append it to the data list
                    if specific_info:
                        data_list.append({'Country': strong_text, 'Type of Sanctions': specific_info})
        else:
            print(f"The content type '{content_type}' is not a PDF.")
 
    except Exception as e:
        print(f"Error: {e}")
 
# Base URL of the website
url = "https://www.securitycouncilreport.org/monthly-forecast/"
driver = webdriver.Chrome()  # You should provide the path to your chromedriver executable if not in PATH
driver.get(url)
 
# Step 2: Get the href from the first element in //*[@id="publication-stack"]/li[1]/div/h4/a
first_link_element = driver.find_element(By.XPATH, '//*[@id="publication-stack"]/li[1]/div/h4/a')
first_link = first_link_element.get_attribute("href")
 
# Step 3: Access the link from href
driver.get(first_link)
 
# Step 4: Get href from all elements in //*[@id="articles"]/li/h4/a
article_links_elements = driver.find_elements(By.XPATH, '//*[@id="articles"]/li/h4/a')
article_links = [element.get_attribute("href") for element in article_links_elements]
 
# Step 5: Loop through each article link and perform the task
for article_link in article_links:
    driver.get(article_link)
 
    # Your task
    text_to_find = "Fact Sheet on Sanctions"  # Replace with the text you're searching for
    highlights_list = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="highlights"]/li'))
    )
 
    # Search for the text in the highlights list
    fact_sheet_found = False
    for highlight in highlights_list:
        li_text = highlight.text
        a_tag = highlight.find_element(By.TAG_NAME, 'a')
        a_text = a_tag.text
        if text_to_find in li_text:
            fact_sheet_found = True
            fact_sheet_href = a_tag.get_attribute('href')
            strong_text = highlight.find_element(By.TAG_NAME, 'strong').text
            data = scrape_content_from_url(fact_sheet_href, start_keyword, end_keywords)
 
# Create a DataFrame from the data list
security_council_report_df = pd.DataFrame(data_list)
 
# Ensure all columns are present, fill NaN for missing columns
security_council_report_df = security_council_report_df.reindex(columns=columns_list, fill_value=None)
 
# Display the DataFrame
print(security_council_report_df)

                            Country  Restrictive Measures  Executive Orders  \
0                       Afghanistan                   NaN               NaN   
1                 Counter-Terrorism                   NaN               NaN   
2                              Iraq                   NaN               NaN   
3  Democratic Republic of the Congo                   NaN               NaN   
4                             Yemen                   NaN               NaN   
5                       South Sudan                   NaN               NaN   
6                             Libya                   NaN               NaN   

                                   Type of Sanctions  
0       Arms embargo, travel ban and assets freeze\n  
1   Arms embargo, travel ban and assets freeze.\n...  
2   Assets freeze and arms embargo.\nSanctions me...  
3   Arms embargo, travel ban and assets freeze.\n...  
4   Assets freeze; travel ban; targeted arms emba...  
5   Travel ban and assets freeze (as 

In [15]:
display(security_council_report_df)

,Country,Restrictive Measures,Executive Orders,Type of Sanctions
0,Afghanistan,NaN,NaN,"Arms embargo, travel ban and assets freeze\n"
1,Counter-Terrorism,NaN,NaN,"Arms embargo, travel ban and assets freeze.\n..."
2,Iraq,NaN,NaN,Assets freeze and arms embargo.\nSanctions me...
3,Democratic Republic of the Congo,NaN,NaN,"Arms embargo, travel ban and assets freeze.\n..."
4,Yemen,NaN,NaN,Assets freeze; travel ban; targeted arms emba...
5,South Sudan,NaN,NaN,Travel ban and assets freeze (as set out in r...
6,Libya,NaN,NaN,"Arms embargo, travel ban, assets freeze, meas..."


COLOUMN 2

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time

# Path to your ChromeDriver
chrome_driver_path = 'C:\\Users\\LENOVO\\OneDrive\\Desktop\\chromedriver-win64\\chromedriver.exe'

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Run in headless mode

service = Service(chrome_driver_path)
service.start()

# Start a browser session
driver = webdriver.Chrome(service=service, options=chrome_options)

url = 'https://www.sanctionsmap.eu/#/main'
driver.get(url)

In [17]:
ul_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'filter-list'))
)

In [18]:
columns_list = ["Country", "Restrictive Measures","Executive Orders","Type of Sanctions"]
main_df = pd.DataFrame(columns=columns_list)
main_df['Executive Orders'] = None
main_df['Type of Sanctions'] = None
display(main_df)

,Country,Restrictive Measures,Executive Orders,Type of Sanctions


In [19]:
for ul in ul_elements:
    country_element = ul.find_element(By.CSS_SELECTOR, 'li[data-heading="Country or Category"] div a')
    country_text = country_element.text
#     print("\n Country:", country_text)
    
    restrictiveMeasure_element = ul.find_element(By.CSS_SELECTOR, 'li[data-heading="Restrictive measures"]')
    child_li_elements = restrictiveMeasure_element.find_elements(By.TAG_NAME, 'li')
    
    res_measure_text = ""
    
    for index ,li_ele in enumerate(child_li_elements):
        div_element = li_ele.find_element(By.TAG_NAME, 'div')
        div_element.click()

        try:
            restrictiveMeasure_div_element = driver.find_element(By.CLASS_NAME, 'popover-content')
            Restrictive_measure_text = restrictiveMeasure_div_element.text
            res_measure_text = res_measure_text + f"\n● {Restrictive_measure_text}"
            

        except NoSuchElementException:
            print("Div with class _ngcontent-c5 not found")

    row_object = {"Country": country_text , "Restrictive Measures": res_measure_text}
    main_df = pd.concat([main_df, pd.DataFrame([row_object])], ignore_index=True)
    
    
main_res_df = main_df.groupby('Country')['Restrictive Measures'].agg(lambda x: '\n'.join(x)).reset_index()
main_res_df['Executive Orders'] = None
main_res_df['Type of Sanctions'] = None
display(main_res_df)


,Country,Restrictive Measures,Executive Orders,Type of Sanctions
0,Afghanistan,\n● It is prohibited to export arms and relate...,None,None
1,Belarus,\n● It is prohibited to export arms and relate...,None,None
2,Bosnia & Herzegovina,\n● There are no persons listed under this res...,None,None
3,Burundi,\n● All assets of the listed persons and entit...,None,None
4,Central African Republic,\n● It is prohibited to export arms and relate...,None,None
5,Chemical weapons,\n● All assets of the listed persons and entit...,None,None
6,China,\n● Political declaration from 1989 by the Mem...,None,None
7,Cyber-attacks,\n● All assets of the listed persons and entit...,None,None
8,Democratic People's Republic of Korea (DPRK – ...,\n● It is prohibited to export the following i...,None,None
9,Democratic Republic of the Congo,\n● It is prohibited to export arms and relate...,None,None


In [69]:
merged_df = pd.concat([main_res_df,df, security_council_report_df], ignore_index=True)

In [70]:
display(merged_df)

,Country,Restrictive Measures,Executive Orders,Type of Sanctions
0,Afghanistan,\n● It is prohibited to export arms and relate...,None,None
1,Belarus,\n● It is prohibited to export arms and relate...,None,None
2,Bosnia & Herzegovina,\n● There are no persons listed under this res...,None,None
3,Burundi,\n● All assets of the listed persons and entit...,None,None
4,Central African Republic,\n● It is prohibited to export arms and relate...,None,None
...,...,...,...,...
192,Iraq,NaN,NaN,Assets freeze and arms embargo.\nSanctions me...
193,Democratic Republic of the Congo,NaN,NaN,"Arms embargo, travel ban and assets freeze.\n..."
194,Yemen,NaN,NaN,Assets freeze; travel ban; targeted arms emba...
195,South Sudan,NaN,NaN,Travel ban and assets freeze (as set out in r...


In [71]:
new = pd.merge(merged_df,df_countries[['Country']], on='Country', how='inner')

In [72]:
display(new)

,Country,Restrictive Measures,Executive Orders,Type of Sanctions
0,Afghanistan,\n● It is prohibited to export arms and relate...,None,None
1,Afghanistan,None,\n•14064 Protecting Certain Property of Da Afg...,None
2,Afghanistan,NaN,NaN,"Arms embargo, travel ban and assets freeze\n"
3,Belarus,\n● It is prohibited to export arms and relate...,None,None
4,Belarus,None,\n•13224 Blocking Property and Prohibiting Tra...,None
...,...,...,...,...
63,Burma,None,\n•https://ofac.treasury.gov/media/6181/downlo...,None
64,Cuba,None,\n•13405 Blocking Property of Certain Persons ...,None
65,Ethiopia,None,\n•14032 - Addressing the Threat from Securiti...,None
66,Hong Kong,None,\n•Beltran Leyva Organization - December 2023 ...,None


In [74]:
final = new.groupby('Country')[['Restrictive Measures', 'Executive Orders', 'Type of Sanctions']].agg(lambda x: ' '.join(str(val) for val in x if pd.notna(val))).reset_index()

In [75]:
display(final)

,Country,Restrictive Measures,Executive Orders,Type of Sanctions
0,Afghanistan,\n● It is prohibited to export arms and relate...,\n•14064 Protecting Certain Property of Da Afg...,"Arms embargo, travel ban and assets freeze\n"
1,Balkans,,\n•13268 Termination of Emergency With Respect...,
2,Belarus,\n● It is prohibited to export arms and relate...,\n•13224 Blocking Property and Prohibiting Tra...,
3,Bosnia & Herzegovina,\n● There are no persons listed under this res...,,
4,Burma,,\n•https://ofac.treasury.gov/media/6181/downlo...,
5,Burundi,\n● All assets of the listed persons and entit...,,
6,Central African Republic,\n● It is prohibited to export arms and relate...,\n•14033 Blocking Property and Suspending Entr...,
7,China,\n● Political declaration from 1989 by the Mem...,,
8,Cuba,,\n•13405 Blocking Property of Certain Persons ...,
9,Democratic People's Republic of Korea (DPRK – ...,\n● It is prohibited to export the following i...,,


In [26]:
final.to_excel('final.xlsx', index=False)

IMPORTING DATA TO ORACLE DB

In [2]:
pip install cx_Oracle

Note: you may need to restart the kernel to use updated packages.
